# Common Name - Summary Code
Author: 

In [1]:
%matplotlib inline
import sqlite3
import pprint
import pandas as pd
#import geopandas as gpd
pd.set_option('display.width', 600)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.max_rows', 100)
from IPython.display import Image
import repo_functions as functions
from pygbif import occurrences
import matplotlib.pyplot as plt
from datetime import datetime
import os
print(os.getcwd())
t1 = datetime.now()

/Users/nmtarr/Code/NC-BBA


### General Setup  
Fill out the next cell.

In [2]:
summary_id = 'summer1'
gap_id = 'bloshx'
summary_name = 'loshSummer' # a short, memorable name to use for file names etc.
codeDir = "/users/nmtarr/code/NC-BBA/"
projDir = "/users/nmtarr/documents/ncba/"
dataDir = projDir + "data/"
inDir = dataDir + "inputs/"
outDir = dataDir + "outputs/"
request_id = 'GBIFr19'
filter_id = 'GBIFf8'
summary_db = outDir + summary_name + ".sqlite"
NChucs = inDir + "NC_hucs"
NCBAblocks = inDir + "ncba_blocks"
NCcounties = inDir + "NC_counties"

In [3]:
notebook_name = gap_id + summary_id + ".ipynb"
summary_db = outDir + gap_id + summary_id + '.sqlite'
# path to parameters.sqlite within occurrence record wrangler repo
%cd /Users/nmtarr/Code/occurrence-records-wrangler/
print(os.getcwd())
orw_db = "parameters.sqlite"
connection = sqlite3.connect(orw_db)
cursor = connection.cursor()

orw_id = cursor.execute("SELECT species_id FROM species_concepts WHERE gap_id = ?;", (gap_id,)).fetchone()[0]
occ_db = outDir + '/{0}{1}{2}.sqlite'.format(orw_id, request_id, filter_id) # Path to occurrence db created from occurrence records wrangler db
parameters_db = '/users/nmtarr/code/nc-bba/evaluations.sqlite' # path to evaluations db

common_name = connection.execute("SELECT common_name FROM species_concepts WHERE gap_id = ?;", (gap_id,)).fetchone()[0]
sci_name = connection.execute("SELECT scientific_name FROM species_concepts WHERE gap_id = ?;", (gap_id,)).fetchone()[0]
shucs_loc = '/users/nmtarr/data/SHUCS'

/Users/nmtarr/Code/occurrence-records-wrangler
/Users/nmtarr/Code/occurrence-records-wrangler


OperationalError: no such table: species_concepts

## Summary Parameters

Summary parameters need to be set and justified in the cells within this section.  Values that are entered here will be used to update cells within the 'evaluations' table stored in evaluations.sqlite. The decisions about what values to use are primarily documented here, not in the evaluations database.

Note that the evaluation ID and species' GAP code are set in the cell above, not below.  I am proposing that evaluation parameter sets also be documented as unique entities in a database (i.e, evaluations.sqlite).  Each evaluation can be given a unique id that can be used in documentation, file naming, and for the names of the columns that will be added to the GAP range table to record the results of the evaluation.  In this example, the evaluation_id is __summer1__.

### Filter Sets

In [ ]:
filter_sets = "{0}, {1}".format(request_id, filter_id)

### Years
Justification: A five year period ending this year to simulate an atlas duration.

In [ ]:
years = "2014, 2015, 2016, 2017, 2018, 2019"

### Months
Justification: Safe summer months

In [ ]:
months = "4,5,6,7"

#### Minimum Count
Justification: This species is fairly easy to identify, but misidentification could be possible.

In [ ]:
min_count = 2

#### Error Tolerance
Justification: Chosen for demonstration purposes.

In [ ]:
error_tolerance = 20

### Credits

In [ ]:
creator = "N. Tarr"
date = datetime.now()

### Justification

In [ ]:
justification = "See " + notebook_name

### Notes

In [ ]:
notes = """This was built for proof of concept"""

### Write to summary_params.sqlite

In [ ]:
connjup = sqlite3.connect(parameters_db)
cursorjup = connjup.cursor()

# Make a row for species-evaluation
sqlrow = """INSERT OR IGNORE INTO evaluations ("evaluation_id", "species_id") VALUES (?, ?);"""
vals = [eval_id, gap_id]
cursorjup.execute(sqlrow, vals)

# Filter sets
sqlfilters = """UPDATE evaluations SET filter_sets=? WHERE evaluation_id=? AND species_id=?;"""
vals = [filter_sets, eval_id, gap_id]
cursorjup.execute(sqlfilters, vals)

# Years
sqlyear = """UPDATE evaluations SET years=? WHERE evaluation_id=? AND species_id=?;"""
vals = [years, eval_id, gap_id]
cursorjup.execute(sqlyear, vals)

# Months
sqlmonths = """UPDATE evaluations SET months=? WHERE evaluation_id=? AND species_id=?;"""
vals = [months, eval_id, gap_id]
cursorjup.execute(sqlmonths, vals)

# Evaluation Method
sqlmethod = """UPDATE evaluations SET method=? WHERE evaluation_id=? AND species_id=?"""
vals = [method, eval_id, gap_id]
cursorjup.execute(sqlmethod, vals)

# Minimum Count
sqlmin = """UPDATE evaluations SET min_count=? WHERE evaluation_id=? AND species_id=?"""
vals = [min_count, eval_id, gap_id]
cursorjup.execute(sqlmin, vals)

# Error Tolerance
sqltolerance = """UPDATE evaluations SET error_tolerance=? WHERE evaluation_id=? AND species_id=?"""
vals = [error_tolerance, eval_id, gap_id]
cursorjup.execute(sqltolerance, vals)

# Justification
sqljust = """UPDATE evaluations SET justification=? WHERE evaluation_id=? AND species_id=?"""
vals = [justification, eval_id, gap_id]
cursorjup.execute(sqljust, vals)

# Credits
sqlcreator = """UPDATE evaluations SET creator=? WHERE evaluation_id=? AND species_id=?"""
vals = [creator, eval_id, gap_id]
cursorjup.execute(sqlcreator, vals)

# Notes
sqlnotes = """UPDATE evaluations SET notes=? WHERE evaluation_id=? AND species_id=?"""
vals = [notes, eval_id, gap_id]
cursorjup.execute(sqlnotes, vals)

sqldate= """UPDATE evaluations SET date=? WHERE evaluation_id=? AND species_id=?"""
vals = [date, eval_id, gap_id]
cursorjup.execute(sqldate, vals)

connjup.commit()

### Evaluation Parameters
Display the record that was just written.

In [ ]:
df1 = pd.read_sql_query(sql="SELECT * FROM evaluations WHERE evaluation_id='{0}' AND species_id='{1}'".format(eval_id, gap_id), con=connjup)
print(df1.loc[0])

### Request Filter Set

In [ ]:
filterconn = sqlite3.connect(orw_db)
df2 = pd.read_sql_query(sql="SELECT * FROM gbif_requests WHERE request_id='{0}'".format(request_id), con=filterconn)
print(df2.loc[0])

### Post Request Filter Set

In [ ]:
df2 = pd.read_sql_query(sql="SELECT * FROM gbif_filters WHERE filter_id = '{0}'".format(filter_id), con=filterconn)
print(df2.loc[0])
filterconn.close()

# Occurrence Record Retrieval and Display
This repo is dependent upon the occorrence-records-wrangler repo because occurrence data is retrieved here from sqlite occurrence databases generated with the records wrangler repo.  In this section, a connection is established and records are filtered according to the evaluation parameters.  Keep in mind that the occurrence record databases were themselves created with filters so you have to be mindful of how parameters set here compare to ones set there. For example, dates of records included will be determined by whichever process had a more restrictive date range. 

The first step in using occurrence records to evaluate GAP range is to build a database to hold the GAP 12 digit HUCs, range for the species, and suitable occurrence records.  The database is also suitable for performing the necessary spatial queries.  The GAP range is retrieved from ScienceBase and the HUCs would be too if they were available as a shapefile.  

In [ ]:
gap_range = functions.download_GAP_range_CONUS2001v1(gap_id=gap_id, toDir=inDir)

In [ ]:
functions.make_summary_db(summary_db=summary_db, gap_id=gap_id, inDir=inDir,
                          outDir=outDir, NChucs=NChucs, NCBAblocks=NCBAblocks,
                          NCcounties=NCcounties)

Copy records to the evaluation database, filtering out records from months or years that aren't wanted.

In [ ]:
functions.occurrence_records_to_db(occurrence_db, summary_db, years, months)

### Display Occurrence Points and GAP Range Map
A presence shapefile must be created for display because the sciencebase range shapefiles are seasonal, which is not the focus here.

Display occurrence records over GAP range

In [ ]:

gap_range2 = "{0}{1}_presence_4326".format(outDir, gap_id)

shp1 = {'file': gap_range2, 'column': None, 'alias': 'GAP range map - presence',
        'drawbounds': False, 'linewidth': .5, 'linecolor': 'm',
        'fillcolor': 'm', 'marker':'s'}

shp2 = {'file': '{0}{1}_circles'.format(outDir, summary_name), 'column': None,
        'alias': 'Occurrence records', 'drawbounds': True, 'linewidth': .75, 'linecolor': 'k',
        'fillcolor': None, 'marker':'o'}

# Display occurrence polygons
title="{1} ({0})".format(years, sci_name)
functions.MapShapefilePolygons(map_these=[shp1, shp2], title=title)


# GAP Known Range Data Evaluation
With all the data in a sqlite database with spatialite capabilities, we can perform the evaluation.

In [ ]:
functions.evaluate_GAP_range(eval_id=eval_id, gap_id=gap_id, eval_db=eval_db,
                             outDir=outDir, codeDir=codeDir)

In [ ]:
connr = sqlite3.connect(eval_db)
df4 = pd.read_sql_query(sql="SELECT strHUC12RNG AS HUC12RNG, "
                                    "intGAPOrigin AS Origin, intGAPPresence AS Presence, "
                                    "intGAPReproduction AS Reproduction,"
                                    "intGAPSeason AS Season, eval_cnt, eval, "
                                    "validated_presence AS validated_pres FROM sp_range WHERE eval_cnt >=0", con=connr)
df4.set_index(["HUC12RNG"], inplace=True)
print("Tabular results of the evaluation")
print(df4)

In [ ]:
print("Mapped results of the evaluation.")
shp3 = {'file': '{0}{1}_eval'.format(outDir, gap_id), 'column': 'eval',
        'alias': 'eval', 'column_colors': {1: 'b', 0: 'r'}, 
        'value_alias': {1:'Agreement', 0:'Disagreement'}, 'drawbounds': False, 
        'marker': "s"}
title="{0} -- {1}".format(common_name, eval_id)
functions.MapShapefilePolygons(map_these=[shp1, shp3], title=title)

In [ ]:
dups0 = connr.execute("SELECT COUNT(occ_id) FROM evaluation_occurrences GROUP BY geom_xy4326, occurrenceDate;").fetchall()
dups1 = [x[0] for x in dups0]
dups2 = [x for x in dups1 if x > 1]
print(str(len(dups2)) + ' records were duplicates based on xy coordinate and date-time')

After occurrence circles are attributed to HUCs, the results can be recorded in the species' range map table in terms of whether the two data sets agreed and whether they validate the GAP range data for any HUCs. For each evaluation, a column is added for 1) how many records could be attributed to each huc and 2) whether there is agreement at that huc (1 for yes, 0 for no, 'None' for no data for that huc) and 3) whether the GAP range has been validated by the evaluation.

# Summary of Results

### How many records were available in the occurrence database?

In [ ]:
count = connr.execute("SELECT COUNT(occ_id) FROM evaluation_occurrences;").fetchone()[0]
print(str(count) + " occurence records were suitable for this evaluation of the range.")

### How many of the records were attributable to a HUC?

In [ ]:
hucable = connr.execute("SELECT SUM(eval_cnt) FROM sp_range WHERE eval_cnt >=0").fetchall()[0]
print(str(hucable[0]) + " records were attributable to a HUC.")

### How many hucs had records attributed to them?

In [ ]:
containers = connr.execute("SELECT COUNT(eval_cnt) FROM sp_range WHERE eval_cnt >=0").fetchall()[0]
print(str(containers[0]) + " HUCs 'contained' records.")

### How many records were not used because of the minimum count?

In [ ]:
ones = connr.execute("SELECT SUM(eval_cnt) FROM sp_range WHERE eval_cnt < ?", (min_count,)).fetchall()[0]
if ones[0] != None:
    print(str(ones[0]) + " HUCs had occurrences but were not validated because they didn't meet the minimum.")
else:
    print("None")

### How many HUCs were validated?

In [ ]:
validated = connr.execute("SELECT COUNT(validated_presence) FROM sp_range WHERE eval = 1").fetchall()[0]
print(str(validated[0]) + " HUCs were validated.")

### How many HUCs did GAP appear to omit?

In [ ]:
missed = connr.execute("SELECT COUNT(eval) FROM sp_range WHERE eval = 0".format(eval_id)).fetchall()[0]
print(str(missed[0]) + " HUCs were missed.")

### What was the maximum number of occurrences attributable to a single HUC?

In [ ]:
maxi = connr.execute("SELECT MAX(eval_cnt) FROM sp_range").fetchall()[0]
print("The maximum number of records attributed to a HUC was " + str(maxi[0]))

### Runtime

In [ ]:
t2 = datetime.now()
print(t2 - t1)

# Next Steps
This is just a starting point that needs scrutiny.  It is currently hard-coded for a single species, so deploying it would require redesigning to accomodate large numbers of species, multiple users, many more occurrence records, optimal methods for evaluation and range delineation among other things.  